In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

In [16]:
instruction_seb_1 = """You are information security proffesional expert. 
    Your goal is to perform DEEP analysis if provided link is secure and can be 
    accessed by user safely. 
    Open provided link and check if domain and page content are save.
    Check if webpage is malicous and if its purpuse is to cheat people  
    forcing them to provide personal information such as address, credit card etc. 
    Access 3 known security sites to see if there is already some feedback available about the url or its domain.
    Please recommend with links best sites that can verify if the url is save"""

instruction_seb_2 = """You are information security proffesional expert. 
    Your goal is to perform BASIC analysis if provided link is secure and can be 
    accessed by user safely. 
    Open provided link and check if domain and page content are save.
    Check if webpage is malicous and if its purpuse is to cheat people  
    forcing them to provide personal information such as address, credit card etc. 
    Access 1 known security site to see if there is already some feedback available about the url or its domain.
    Please recommend with links best sites that can verify if the url is save"""

In [17]:
security_agent_seb_1 = Agent(
    name="Deep proffessional expert", 
    instructions=instruction_seb_1,
    model="gpt-4o-mini"
    )

security_agent_seb_2 = Agent(
    name="Basic proffessional expert", 
    instructions=instruction_seb_1,
    model="gpt-4o-mini"
    )

In [18]:
description = "Check if link is valid from security perspective"

securitycheck_1 = security_agent_seb_1.as_tool(tool_name="professional_checker",
   tool_description=description
   )
securitycheck_2 = security_agent_seb_2.as_tool(tool_name="basic_checker",
   tool_description=description
   )

In [19]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body"""
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("wieczoreks@hotmail.com")  # Change to your verified sender
    to_email = To("wieczorekseb@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [27]:
subject_instructions = "You can write email subject from a given email body (in the field of IT software security feedback)"
subject_writer = Agent(
    name="subject_writer",
    instructions=subject_instructions, 
    model="gpt-4o-mini")
subject_writer_tool = subject_writer.as_tool(
    tool_name="subject_writer_tool",
    tool_description="Write a subject for email for security check on url")

html_formatter_instructions ="You can convert given email body text about url security feedback into HTML email body\
    Your goal is to convert given email body text to simple nice looking html email layout \
        Make sure on top of the email, below greeting to write link to investigated url"
html_formatter=Agent(
    name="html_formatter_tool",
    instructions=html_formatter_instructions,
    model="gpt-4o-mini"
    )
html_formatter_tool=html_formatter.as_tool(
    tool_name="html_formatter_tool",
    tool_description="Convert a text email body to an HTML email body")

In [21]:
email_tools = [subject_writer_tool, html_formatter_tool, send_html_email]

In [22]:
email_creation_instructions = "You are an email formatter and sender.\
    You receive body of the email (security feedback abou url) to be sent.\
    You first use subject_writer_tool to write email subject then use \
    html_formatter_tool to convert the body to HTML.\
    and you use the send_html_email tool to send the email with the subject and HTML body formatted for email.\
    Make sure not to send pure HTML in the email that when received looks like a HTML code instead of formattd email"
email_agent = Agent(
    name="Email Manager", 
    instructions=email_creation_instructions,
    model="gpt-4o-mini",
    tools=email_tools,
    handoff_description="Convert an email to HTML and send it"
    )

In [32]:
url= "https://www.reddit.com/r/okbuddycinephile/comments/1otx0q8/favorite_actor_or_actress_who_found_success/"

In [24]:
security_tools = [securitycheck_1, securitycheck_2]
handoff_mailer = [email_agent]  

In [ ]:
security_manager_instructions = """You are security manager in the field of IT software security
      Follow these steps carefuly:
      1. Generate two security feedbacks about provided url_1 
        Use security_tools: securitycheck_1 & securitycheck_2 tools to generate 2 different security feedbacks about provided url
      2. Evaluate and select
        Review feedbacks and choose single best feedback using your judgement of which one is most effective.
        You can use the tools only once.
      3. Handoff for Sending
        Pass only winning feedback to the 'Email Manager' agent
Crucial Rules:
- You must use the security_tools tools to generate feedbacks — do not write them yourself.
- You must hand off exactly ONE feedback security text to the Email Manager — never more than one.
"""

security_manager = Agent(
    name="Security Manager",
    instructions=security_manager_instructions,
    model="gpt-4o-mini",
    tools=security_tools,
    handoffs=handoff_mailer
)

message="Pick best security advise about url:"+ url + " and send email with security feedback"


In [30]:
with trace("Pick best security advise about url and send email"):
    result = await Runner.run(security_manager, message)